# Notebook 5.3 - Curation Keywords: Remove Unused

[From a comment to the MarketPlace Curation issue #1](https://github.com/SSHOC/marketplace-curation/issues/1):  
...

To solve this, we need another routine that does the cleanup of the sshoc-keyword-vocabulary concepts:

- Go through all concepts of the vocabulary keywords => GET /api/vocabularies/sshoc-keyword (this is a case-sensitive list)
- Look, if the keyword is not connected to any item => GET /api/item-search and search in the result of this call if in the object "facets": {"keyword": ...} the concept shows up (beware, case-sensitive)
- If the concept is not there (it shows only concepts that are connected to one or more items) this means, we can delete it with calling DELETE /api/vocabularies/sshoc-keyword/concepts/{code} (the code needs to come from the first call) 

...

## 0 Requirements to run this notebook

This section gives all the relevant information to "interact" with the MP data.

### 0.1 libraries
*There are a number of external libraries needed to run the notebook* 

*Furthermore, a dedicated SSH Open Marketplace library - sshmarketplacelib - with customised functions has been created and can be imported using the python import commands.* 

*Below the libraries import needed to run this notebook*

In [ ]:
import numpy as np
import pandas as pd
import requests
#import the MarketPlace Library 
from sshmarketplacelib import MPData as mpd
from sshmarketplacelib import  eval as eva, helper as hel
import os
import json
import yaml
import urllib.request
import urllib
import errno

### 0.1.1 Download data from configuration file

In [ ]:
if (os.path.isfile('config.yaml')):
            configfile="config.yaml"
else:
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), "config.yaml")
try:
    with open(configfile, 'r') as stream:
        try:
            conf=yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
except FileNotFoundError:
    print('Warning config.yaml file not present! Please create it and set the values, store it in the main directory')
debugMode=conf['DEBUG']

In [ ]:
if debugMode:
    print('Running in debug mode')
else:
    print('Running in production mode, updated data will be written back to the server')

### 0.2 Functions

In [ ]:
def getMPSSHOCKeywords():
    df_keywords = pd.DataFrame()
    items= pd.DataFrame()
    myurl=conf['API']['SERVER']+conf['DATASET_ENTRYPOINTS']["sshockeywords"]
        
    with urllib.request.urlopen(myurl+'?perpage=20') as url:
        df_desc_par = json.load(url)
        
    df_keywords= pd.DataFrame(df_desc_par["conceptResults"]['concepts'])
    df_keywords['var']=1
    start=1
        
    if not df_keywords.empty:
        pages=df_desc_par["conceptResults"]['pages']
        start+=1
        
        mdx = pd.Series(range(start, pages+1))
        for var in mdx:
            turl = myurl+"?page="+str(var)
            #print (f'{var}/{pages}, {df_keywords.shape[0]}, {turl+"&perpage=20"}')
            try:
                with urllib.request.urlopen(turl+'&perpage=20') as murl:
                    #print (f'{var} - {turl+"&perpage=100"}')
                    df_desc_par = json.load(murl)
                    temp=pd.DataFrame(df_desc_par["conceptResults"]["concepts"])
                    temp['var']=var
                    df_keywords=pd.concat([df_keywords, temp])
                    #df_keywords=pd.concat([df_keywords, pd.DataFrame(df_desc_par["conceptResults"]["concepts"])])
            except:
                print(f'SEVERE: Error getting keywords. (Error loading {turl})')
    return df_keywords

### 0.2 Get the data



In [ ]:
mpdata = mpd()
df_tool_flat =mpdata.getMPItems ("toolsandservices", True)
df_publication_flat =mpdata.getMPItems ("publications", True)
df_trainingmaterials_flat =mpdata.getMPItems ("trainingmaterials", True)
df_workflows_flat =mpdata.getMPItems ("workflows", True)
df_datasets_flat =mpdata.getMPItems ("datasets", True)

In [ ]:
#df_tool_flat[['description', 'persistentId']].to_csv(path_or_buf='ts_peiddescr.csv')

In [ ]:
# df2 = df_tool_flat.pivot_table(index = ['persistentId'], aggfunc ='size')
# df_2=df2.to_frame()
# df_2.reset_index(inplace=True)
# df_3=df_2.rename(columns={df_2.columns[1]:'test', df_2.columns[0]:'myid'})
# df_3.sort_values(['test', 'myid'], ascending=False).tail(10)

In [ ]:
utils=hel.Util()
resultfields=['persistentId', 'MPUrl', 'category', 'label', 'status', 'type.code', 'type.label', 'concept.code', 'concept.label', 'concept.uri', 'concept.vocabulary.scheme']
udf_alprop=utils.getAllPropertiesBySources()
udf_alprop=udf_alprop.loc[ : ,resultfields]

In [ ]:
udf_alprop.head()

In [ ]:
# udf_alprop_nan=udf_alprop[udf_alprop['concept.code'].isnull()]
# udf_alprop_nan.count()

In [ ]:
udf_alprop_new = udf_alprop.drop_duplicates(subset = ['concept.code'],keep = 'last').reset_index(drop = True)
udf_alprop_new.count()

In [ ]:
df_sshockeywords=getMPSSHOCKeywords()


In [ ]:
# df_sshockeywords.sort_values('code').head()
df2 = df_sshockeywords.pivot_table(index = ['code'], aggfunc ='size')
df_2=df2.to_frame()
df_2.reset_index(inplace=True)
df_3=df_2.rename(columns={df_2.columns[1]:'test', df_2.columns[0]:'myid'})
df_3.sort_values(['myid', 'test'], ascending=False).iloc[0:10]

In [ ]:
newdf = df_sshockeywords.drop_duplicates(subset = ['code'],keep = 'last').reset_index(drop = True)
newdf.sort_values(['code']).iloc[0:4]

In [ ]:
df_sshockeywords['mycode']=df_sshockeywords['code'].apply(lambda y: " ".join(y.split()).lower().strip().replace('+',' '))
newdf['mycode']=newdf['code'].apply(lambda y: " ".join(y.split()).lower().strip().replace('+',' '))
udf_alprop_new['mycode']=udf_alprop_new['concept.code'].apply(lambda y: " ".join(str(y).split()).lower().strip().replace('+',' '))

In [ ]:
newdf.count()

In [ ]:
df_unusedkeywords1=pd.DataFrame(columns = ['UnusedKW'])
df_usedkeywords1=pd.DataFrame(columns = ['UsedKW'])
for rown, row in newdf.iterrows():
    temp_df=udf_alprop_new[udf_alprop_new['mycode']==row['mycode']]
    #temp_df=udf_alprop_new[(udf_alprop_new['mycode']==row['mycode']) & (udf_alprop_new['status']=='approved')]
    if(not len(temp_df)):
        d = pd.DataFrame(data={'UnusedKW': [row.code]})
        d['label'] = row.label
        d['URI'] = row.uri
        d['concept_code']=row['code']
        df_unusedkeywords1=pd.concat([df_unusedkeywords1, d])
    else:
        #print (f'{row.code}, {temp_df["concept.code"]}')
        du = pd.DataFrame(data={'UsedKW': [row.code]})
        du['label'] = row.label
        du['URI'] = row.uri
        du['itempid']=temp_df.iloc[0].persistentId
        du['itemlabel']=temp_df.iloc[0].label
        
        df_usedkeywords1=pd.concat([df_usedkeywords1, du])
    
    

In [ ]:
df_unusedkeywords1.count()

In [ ]:
df_unusedkeywords1.sort_values('UnusedKW').iloc[0:10]

In [ ]:
df_unusedkeywords1.to_csv(path_or_buf='unusedkw1.csv')

In [ ]:
df_usedkeywords1.count()

In [ ]:
df_usedkeywords1.to_csv(path_or_buf='usedkw.csv')

In [ ]:
import urllib.parse
bearer=mpdata.checkCredentials()
put_headers = {'Content-type': 'application/json', 'Authorization':bearer}
df_log=pd.DataFrame()
MPAPIserver=conf['API']['SERVER']
for unkeyn, unkeyrow in df_unusedkeywords1.iterrows():
    dl = pd.DataFrame(data={'UnusedKW': [unkeyrow.UnusedKW]})
    deleteurl=f'{MPAPIserver}/api/vocabularies/sshoc-keyword/concepts/{urllib.parse.quote(unkeyrow.UnusedKW)}?force=true'
    #deleteurl='https://marketplace-api.sshopencloud.eu/api/vocabularies/sshoc-keyword/concepts/'+urllib.parse.quote(unkeyrow.UnusedKW)+'?force=true'
    print ('#############################')
    print (f'{unkeyrow.UnusedKW}, {unkeyrow.concept_code}')
   
    print (f'{deleteurl} \n')
    print ('------- \n')
    if (not debugMode):
        delete_result=requests.delete(deleteurl, headers=put_headers)
        print (f'{delete_result.status_code}, {delete_result.text}')
        if (delete_result.status_code==404):
            print (' Error deleting keyword')
        #dl['key'] = unkeyrow.UnusedKW
        dl['DeleteResultCode'] = delete_result.status_code
        dl['DeleteResultTest']= delete_result.text
        df_log=pd.concat([df_log, dl])
    else:
        print( 'Running in debug mode, delete command not executed')

# deleteurl='https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/vocabularies/sshoc-keyword/concepts/AAH?force=true'
        


In [ ]:
df_log.groupby('DeleteResultCode').count()

In [ ]:
df_log.to_csv(path_or_buf='conceptsdeletelogforce_9.csv')
#df_log.to_pickle('conceptsdeletelog.pickle')

In [ ]:
df_log.sort_values('DeleteResultCode').tail()